**Import necessary libraries**

In [1]:
import pandas as pd
import re
import numpy as np

Create a `df` from the extracted data

In [2]:
df = pd.read_csv('cars.csv')

Removing unnecessary columns:

1. المجموع ,التكميلي (جسم المركبة) ,طرف ثالث => Insurance information

2. أصحاب سابقون => For previous owners, and it doesn't affect on the price in most cases

3. لون السيارة => The color of the car doesn't affect on the price in most cases (just if it's only `Black` or `White`) so I dropped the column.

4. معروضة => Shown as for sale or change, and this doesn't affect the price

In [3]:
df.drop(columns=['المجموع','التكميلي (جسم المركبة)','طرف ثالث','أصحاب سابقون','لون السيارة','معروضة'],inplace=True)

In the `إضافات` column, I would only look for `Sunroof` addition, so if the adds have a sunroof I will make it 1, if not it will be 0

In [4]:
df['Sunroof'] = df['إضافات'].str.contains('فتحة سقف')
df['Sunroof'] = df['Sunroof'].astype(bool).astype(int)
df.drop('إضافات', axis=1, inplace=True)

Check the missing values, so I can drop the columns that has 70% or more of missing values

In [5]:
missing_percentage = (df.isnull().mean() * 100).round(2)
missing_data_info = pd.DataFrame({'Column': df.columns, 'Missing Percentage': missing_percentage})

missing_data_info

,Column,Missing Percentage
Name,Name,0.00
Model,Model,0.00
Price,Price,0.00
نوع الوقود,نوع الوقود,0.00
أصل السيارة,أصل السيارة,0.00
رخصة السيارة,رخصة السيارة,0.00
نوع الجير,نوع الجير,0.00
الزجاج,الزجاج,0.00
قوة الماتور,قوة الماتور,0.00
عداد السيارة,عداد السيارة,28.28


Drop the columns that has 70% or more of missing values

In [6]:
threshold = 0.7 * len(df)
df.dropna(axis=1, thresh=threshold, inplace=True)

Change the columns names

In [7]:
df.rename(columns={
'أصل السيارة': 'Origin',
'نوع الوقود':'Fuel',
'رخصة السيارة':'License',
'نوع الجير':'Gear',
'الزجاج':'Glass',
'قوة الماتور':'Motor Power',
'عداد السيارة':'Odometer Reading',
'عدد الركاب':'Passengers',
'وسيلة الدفع':'Payment Method'
},inplace=True)

Edit the `Passengers` column, so I could have the number of all passengers

In [8]:
def process_and_sum(input_value):
    input_str = str(input_value)

    processed_str = ''.join(char for char in input_str if char.isdigit())

    arabic_to_english = {'٠': '0', '١': '1', '٢': '2', '٣': '3', '٤': '4',
                         '٥': '5', '٦': '6', '٧': '7', '٨': '8', '٩': '9'}
    
    processed_str = ''.join(arabic_to_english.get(char, char) for char in processed_str)

    processed_str = processed_str.replace('0', '')

    if not processed_str:
        return np.nan
    
    result = sum(int(digit) for digit in processed_str)
    return result

df['Passengers'] = df['Passengers'].apply(process_and_sum)

Applying `Feature Engineering` concept

In [9]:
payment_method_dummies = pd.get_dummies(df['Payment Method'])
df = pd.concat([df, payment_method_dummies], axis=1)

fuel_dummies = pd.get_dummies(df['Fuel'])
df = pd.concat([df, fuel_dummies], axis=1)

origin_dummies = pd.get_dummies(df['Origin'])
df = pd.concat([df, origin_dummies], axis=1)

gear_dummies = pd.get_dummies(df['Gear'])
df=pd.concat([df,gear_dummies], axis=1)

df.drop(columns=['Payment Method','Fuel','Origin','Gear'],inplace=True)

In [10]:
glass_mapping = {
    'الكتروني': 1,
    'يدوي': 0
}

df['Electronic Glass'] = df['Glass'].replace(glass_mapping)

df.drop('Glass', axis=1,inplace=True)

In [11]:
license_mapping = {
    'فلسطينية': 1,
    'نمرة صفراء': 0
}

df['Palestinian License'] = df['License'].replace(license_mapping)

df.drop('License', axis=1,inplace=True)

Change the `True` and `False` values into `1` and `0`

In [12]:
df[['نقدا فقط','إمكانية التقسيط',
    'بنزين','ديزل','كهرباء','هايبرد',
    'تأجير','تجاري','تدريب سياقة','حكومي','خصوصي','عمومي',
    'اوتوماتيك','عادي','نصف اوتوماتيك']] = df[['نقدا فقط','إمكانية التقسيط',
                                               'بنزين','ديزل','كهرباء','هايبرد',
                                               'تأجير','تجاري','تدريب سياقة','حكومي','خصوصي','عمومي',
                                               'اوتوماتيك','عادي','نصف اوتوماتيك']].astype(int)

Convert the `Odometer Reading` values

In [13]:
def extract_and_convert(reading):
    if pd.isna(reading):
        return reading
    
    reading_str = str(reading)
    
    pattern = re.compile(r'(\d+)\s*ألف|\s*(\d+)\s*الف')     
    match = pattern.search(reading_str)

    if match:
        if match.group(1):
            digits = match.group(1)
        else:
            digits = match.group(2)
        
        number = int(digits)

        if len(digits) < 4:
            result = number * 1000
        else:
            result = number
        return result
    else:
        return reading
    

df['Odometer Reading'] = df['Odometer Reading'].apply(extract_and_convert)
df['Odometer Reading'] = df['Odometer Reading'].astype(str).str.replace(r'\d', '')
df['Odometer Reading'] = pd.to_numeric(df['Odometer Reading'], errors='coerce')

Drop the duplicates

In [14]:
df.drop_duplicates()

,Name,Model,Price,Motor Power,Odometer Reading,Passengers,Sunroof,إمكانية التقسيط,نقدا فقط,بنزين,...,تجاري,تدريب سياقة,حكومي,خصوصي,عمومي,اوتوماتيك,عادي,نصف اوتوماتيك,Electronic Glass,Palestinian License
0,كيا اوبتيما,2014,100000,2000,75000.0,5.0,1,0,1,1,...,0,0,0,1,0,1,0,0,1,1
1,كيا سورينتو,2007,60000,2500,130000.0,8.0,1,1,0,0,...,0,0,0,1,0,0,0,1,1,1
2,هونداي افانتي,2006,43500,1600,NaN,NaN,0,0,1,1,...,0,0,0,1,0,1,0,0,1,1
3,فيات 127,1982,5500,906,NaN,5.0,0,1,0,1,...,0,0,0,1,0,0,1,0,0,1
4,بيجو 208,2014,54000,1200,38000.0,5.0,0,0,1,1,...,0,0,0,1,0,1,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6859,كيا مورننغ,2015,45000,1000,130000.0,5.0,0,1,0,0,...,0,0,0,1,0,1,0,0,1,1
6860,فورد ترانزيت,2002,48000,2400,0.0,8.0,1,0,1,0,...,0,0,0,0,1,0,1,0,1,1
6861,بيجو بارتنر,2018,87000,1600,50000.0,5.0,0,0,1,0,...,0,0,0,1,0,0,1,0,1,1
6862,كيا سورينتو,2017,126000,2200,100.0,7.0,1,0,1,0,...,0,0,0,1,0,1,0,0,1,1


## Filling the missing values

#### **Odometer Reading**

1. I've filled the missing values with the `mean` of the `Odometer Reading` of the cars that have the same `Name` and `Model`
2. If the car does not exist before, then I filled its `Odometer  reading` with the mean of the cars with the same `Model`.

#### **Passengers**

I've filled the `Passengers` number with the most frequent value (`mode`)

#### **Motor Power**

I've replaced the `Motor Power` based on the `Name` and `Model` with the most frequent value (`mode`)

In [15]:
df['Odometer Reading'] = df.groupby(['Name', 'Model'])['Odometer Reading'].transform(lambda x: x.fillna(x.mean()))
df['Odometer Reading'] = df.groupby('Model')['Odometer Reading'].transform(lambda x: x.fillna(x.mean()))

df['Passengers'] = df['Passengers'].fillna(df['Passengers'].mode()[0])


df['Motor Power'] = df.groupby(['Name', 'Model'])['Motor Power'].transform(lambda x: x.replace(x.mode()))

**Removing Outliers**

In [16]:
def remove_outliers_iqr(df, columns):
    for column in columns:
        Q1 = df[column].quantile(0.25)
        Q3 = df[column].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        df = df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]
    return df

columns_to_remove_outliers = ['Motor Power', 'Odometer Reading']

df = remove_outliers_iqr(df, columns_to_remove_outliers)

Changing the `float` values into `int` values

In [17]:
df['Price'] = df['Price'].astype('int')
df['Odometer Reading'] = df['Odometer Reading'].astype('int')
df['Passengers'] = df['Passengers'].astype('int')

Applying the `Feature Engineering` concept on the `Name` field

Change the (`True / False`) values into `int` values

In [18]:
name_dummies = pd.get_dummies(df['Name'])
df = pd.concat([name_dummies, df], axis=1)
df[name_dummies.columns] = df[name_dummies.columns].astype(int)
df = df.drop('Name', axis=1)

Convert the `Data Frame` into a `.csv` file

In [19]:
csv_file_path = 'cleaned_cars.csv'
df.to_csv(csv_file_path, index=False)